In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [14]:
##Load the trained model,scaler pickle,onehot
from tensorflow.keras.models import load_model
import pickle

# Load the trained model
model = load_model('model.h5')

# Load encoders and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [15]:
##Example Input data
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender': 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'hasCrCard': 1,
    'IsActiveMember' : 1,
    'EstimatedSalary':50000

}

In [16]:
##OneHot Encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\ramsa\OneDrive\Desktop\DL_Project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [17]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,hasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [18]:
##Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,hasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [19]:
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,hasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [20]:
##Scaling the Input data
# Fix the column name to match the scaler's expectation
input_df = input_df.rename(columns={'hasCrCard': 'HasCrCard'})

# Now you can safely scale
input_scaled = scaler.transform(input_df)

In [21]:
##Predict the Model
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0.00854527]], dtype=float32)

In [22]:
prediction_proba = prediction[0][0]

In [23]:
prediction_proba

0.008545272

In [24]:
if(prediction_proba > 0.5):
    print('The customer likely to churn')
else:
    print('Customer not likely to churn')

Customer not likely to churn


In [25]:
import tensorflow as tf
print(tf.__version__)

2.17.1
